In [20]:

import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import random
import pandas as pd

def replace_tokens(target_word,text, tokenised_text, keywords, punctuation_list, mode):
    keyword_replacements = []
    index = 0
    
    target_token = target_word.lower()
    try:
        masked_index = tokenised_text.index(target_token)
    except ValueError:
        return (text, target_word)
        
    tokenised_text[masked_index] = '[MASK]'

    indexed_tokens = tokeniser.convert_tokens_to_ids(tokenised_text)
    segments_ids = [1] * len(tokenised_text)
    segments_ids[0] = 0
    segments_ids[1] = 0

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    # Predict all tokens
    predictions = model(tokens_tensor, segments_tensors)

    chosen_token = "X"

    while True :
        predicted_index = torch.argmax(predictions[0, masked_index]).item()
        predicted_token = tokeniser.convert_ids_to_tokens([predicted_index])[0]
        if predicted_token not in punctuation_list and predicted_token.find(target_token) < 0 and target_token.find(predicted_token) < 0 and len(predicted_token)> 3:
            print("predicted token ",predicted_token)
            try:
                ri = keyword_replacements.index(predicted_token)
            except ValueError:
                ri = -1
            if predicted_token == 'bell':
                print("pt: " + predicted_token)
                print(f'ri: {ri}')
                print(f'kr: {keyword_replacements[ri]}')
                print(f'k: {keywords[ri]}')
                print(f'k: {keywords[i]}')
            if ri < 0 or (ri >= 0 and keywords[i] == keywords[ri]):
                # and keyword_replacements[ri].find(predicted_token) < 0 and predicted_token.find(keyword_replacements[ri]) < 0)
                chosen_word = predicted_token
                print('ri',ri,'i',i,keywords[i],keywords[ri])
                print(f'{target_word} -> {chosen_word}')
                break
            else:
                predictions[0,masked_index,predicted_index] = -9999
        else:
            predictions[0,masked_index,predicted_index] = -9999

    if mode == 'all':
        current_text_tokens = spacy_tokeniser(text)
        text = ' '.join([str(spacy_tokeniser(chosen_word)) if str(token) == target_word else str(token) for token in current_text_tokens])
    else:
        print('before: ',text)
        text = text.replace(target_word, chosen_word, 1)
        print('after:',text)
        index +=1

    tokenised_text = tokeniser.tokenize(text)
    keyword_replacements.append(chosen_word)
    print('keyword_replacements',keyword_replacements)
    return (text,chosen_word)
    

model_path = "bert-base-uncased"
model = BertForMaskedLM.from_pretrained(model_path)
model.eval()

tokeniser = BertTokenizer.from_pretrained(model_path)
a_file = open("data/test_data.txt","r")
lines = a_file.readlines()



f2 = open("data/vocab_new.txt", "r")
keywords = f2.read().split()
shuffled = random.sample(keywords, len(keywords))
#test = random.shuffle(keywords)
keywords = shuffled
'''nlp = English()
spacy_tokeniser = nlp.Defaults.create_tokenizer(nlp)
keywords = spacy_tokeniser(text.replace('dummy. ','',1))'''

punctuation = [
    ',',
    '.',
    '!',
    '?',
    ':',
    ';',
    '...',
    '\n', 
    '\n\n',
    '""',
    '"',
    '\'',
    'with',
    'here',
    'also',
    'more'
    
]

input_df = pd.read_csv('data/QA_set2.csv')
x = False
correct = []
for index, row in input_df.iterrows(): #change the range to get more data points
    #change every other context 
    generated_lines = []
    final_generated_lines = []
    for i in range(1): #((k*10),(k*10) + 10):
        print(row['context'])
        tokenised_text = tokeniser.tokenize(row['context'])
        print(tokenised_text)
        answer = row['answer'].split(' ')
        text = replace_tokens(answer[-1],row['context'], tokenised_text, keywords, punctuation, "selection")
        if text[0]== row['context'] and text[1] == answer[-1]:
            correct.append('yes')
        else:
            correct.append('no')
        if x == True:
            #new_line = (str(text.split('\n')))
            input_df.at[index, 'context'] = text[0]
            x = False
            continue
        else:
            input_df.at[index, 'answer'] = text[1]
            x = True
   
    j += 1
input_df['label'] = correct
input_df.to_csv('output/final_data.csv', index=False)


A woman sells flowers and incense at the market .
['a', 'woman', 'sells', 'flowers', 'and', 'incense', 'at', 'the', 'market', '.']
predicted token  that
ri -1 i 0 irresistibly printer.
woman -> that
before:  A woman sells flowers and incense at the market .
after: A that sells flowers and incense at the market .
keyword_replacements ['that']
A woman sells flowers and incense at the market .
['a', 'woman', 'sells', 'flowers', 'and', 'incense', 'at', 'the', 'market', '.']
predicted token  fruits
ri -1 i 0 irresistibly printer.
incense -> fruits
before:  A woman sells flowers and incense at the market .
after: A woman sells flowers and fruits at the market .
keyword_replacements ['fruits']
A woman sells flowers and incense at the market .
['a', 'woman', 'sells', 'flowers', 'and', 'incense', 'at', 'the', 'market', '.']
predicted token  home
ri -1 i 0 irresistibly printer.
market -> home
before:  A woman sells flowers and incense at the market .
after: A woman sells flowers and incense at t

predicted token  expression
ri -1 i 0 irresistibly printer.
look -> expression
before:  A man with a determined look on his face holds a small wooden hammer while wearing a bib displaying a large red crab , another red crustacean held in his hand while another man sits and holds some food behind him .
after: A man with a determined expression on his face holds a small wooden hammer while wearing a bib displaying a large red crab , another red crustacean held in his hand while another man sits and holds some food behind him .
keyword_replacements ['expression']
A man with a determined look on his face holds a small wooden hammer while wearing a bib displaying a large red crab , another red crustacean held in his hand while another man sits and holds some food behind him .
['a', 'man', 'with', 'a', 'determined', 'look', 'on', 'his', 'face', 'holds', 'a', 'small', 'wooden', 'hammer', 'while', 'wearing', 'a', 'bi', '##b', 'displaying', 'a', 'large', 'red', 'crab', ',', 'another', 'red', 'c

predicted token  mouth
ri -1 i 0 irresistibly printer.
hand -> mouth
before:  A man with a determined look on his face holds a small wooden hammer while wearing a bib displaying a large red crab , another red crustacean held in his hand while another man sits and holds some food behind him .
after: A man with a determined look on his face holds a small wooden hammer while wearing a bib displaying a large red crab , another red crustacean held in his mouth while another man sits and holds some food behind him .
keyword_replacements ['mouth']
A man with a determined look on his face holds a small wooden hammer while wearing a bib displaying a large red crab , another red crustacean held in his hand while another man sits and holds some food behind him .
['a', 'man', 'with', 'a', 'determined', 'look', 'on', 'his', 'face', 'holds', 'a', 'small', 'wooden', 'hammer', 'while', 'wearing', 'a', 'bi', '##b', 'displaying', 'a', 'large', 'red', 'crab', ',', 'another', 'red', 'crust', '##ace', '##a

predicted token  paper
ri -1 i 0 irresistibly printer.
equipment -> paper
before:  A woman works on a piece of computer equipment , while surrounded by a variety of mechanical and computer equipment , in what appears to be some type of laboratory workroom .
after: A woman works on a piece of computer paper , while surrounded by a variety of mechanical and computer equipment , in what appears to be some type of laboratory workroom .
keyword_replacements ['paper']
A woman works on a piece of computer equipment , while surrounded by a variety of mechanical and computer equipment , in what appears to be some type of laboratory workroom .
['a', 'woman', 'works', 'on', 'a', 'piece', 'of', 'computer', 'equipment', ',', 'while', 'surrounded', 'by', 'a', 'variety', 'of', 'mechanical', 'and', 'computer', 'equipment', ',', 'in', 'what', 'appears', 'to', 'be', 'some', 'type', 'of', 'laboratory', 'work', '##room', '.']
predicted token  often
ri -1 i 0 irresistibly printer.
woman -> often
before:  A

predicted token  streets
ri -1 i 0 irresistibly printer.
top -> streets
before:  A male adolescent or young teenager is wearing camouflaged shorts or long swim trunks and is upside down over a body of water like a lake or reservoir whose cement walls and graveled top appear in the corner .
after: A male adolescent or young teenager is wearing camouflaged shorts or long swim trunks and is upside down over a body of water like a lake or reservoir whose cement walls and graveled streets appear in the corner .
keyword_replacements ['streets']
A male adolescent or young teenager is wearing camouflaged shorts or long swim trunks and is upside down over a body of water like a lake or reservoir whose cement walls and graveled top appear in the corner .
['a', 'male', 'adolescent', 'or', 'young', 'teenager', 'is', 'wearing', 'camouflage', '##d', 'shorts', 'or', 'long', 'swim', 'trunks', 'and', 'is', 'upside', 'down', 'over', 'a', 'body', 'of', 'water', 'like', 'a', 'lake', 'or', 'reservoir', 'wh

...

predicted token  people
ri -1 i 0 irresistibly printer.
water -> people
before:  Someone is hang gliding or parachuting down by a large body of water with green hills all around .
after: Someone is hang gliding or parachuting down by a large body of people with green hills all around .
keyword_replacements ['people']
Someone is hang gliding or parachuting down by a large body of water with green hills all around .
['someone', 'is', 'hang', 'gliding', 'or', 'para', '##chu', '##ting', 'down', 'by', 'a', 'large', 'body', 'of', 'water', 'with', 'green', 'hills', 'all', 'around', '.']
predicted token  people
ri -1 i 0 irresistibly printer.
water -> people
before:  Someone is hang gliding or parachuting down by a large body of water with green hills all around .
after: Someone is hang gliding or parachuting down by a large body of people with green hills all around .
keyword_replacements ['people']
A girl standing among a crowd gathered outdoors with the flag of Western Sahara and a white ba

predicted token  dirt
ri -1 i 0 irresistibly printer.
ivy -> dirt
before:  A wall half covered in ivy along with an older two story house line a wet road , the only person on the road is a black wearing a red tank top , black shorts , and light blue sneakers , he 's running a race and sports the number 5003 .
after: A wall half covered in dirt along with an older two story house line a wet road , the only person on the road is a black wearing a red tank top , black shorts , and light blue sneakers , he 's running a race and sports the number 5003 .
keyword_replacements ['dirt']
A wall half covered in ivy along with an older two story house line a wet road , the only person on the road is a black wearing a red tank top , black shorts , and light blue sneakers , he 's running a race and sports the number 5003 .
['a', 'wall', 'half', 'covered', 'in', 'ivy', 'along', 'with', 'an', 'older', 'two', 'story', 'house', 'line', 'a', 'wet', 'road', ',', 'the', 'only', 'person', 'on', 'the', 'road

predicted token  face
ri -1 i 0 irresistibly printer.
shirt -> face
before:  An Asian man wearing a graphic T - shirt that says " Music is the weapon of the future " and an Asian woman in a gray hoodie with hoop earrings sit on a bench and eat a meal of noodles and shrimp from styrofoam plates
after: An Asian man wearing a graphic T - face that says " Music is the weapon of the future " and an Asian woman in a gray hoodie with hoop earrings sit on a bench and eat a meal of noodles and shrimp from styrofoam plates
keyword_replacements ['face']
An Asian man wearing a graphic T - shirt that says " Music is the weapon of the future " and an Asian woman in a gray hoodie with hoop earrings sit on a bench and eat a meal of noodles and shrimp from styrofoam plates
['an', 'asian', 'man', 'wearing', 'a', 'graphic', 't', '-', 'shirt', 'that', 'says', '"', 'music', 'is', 'the', 'weapon', 'of', 'the', 'future', '"', 'and', 'an', 'asian', 'woman', 'in', 'a', 'gray', 'hood', '##ie', 'with', 'hoop', '

predicted token  food
ri -1 i 0 irresistibly printer.
cuisine -> food
before:  Several individuals getting together to play the bagpipes and eat some cuisine .
after: Several individuals getting together to play the bagpipes and eat some food .
keyword_replacements ['food']
An Asian man sits on a stoop polishing cookware .
['an', 'asian', 'man', 'sits', 'on', 'a', 'st', '##oop', 'polish', '##ing', 'cook', '##ware', '.']
predicted token  girl
ri -1 i 0 irresistibly printer.
man -> girl
before:  An Asian man sits on a stoop polishing cookware .
after: An Asian girl sits on a stoop polishing cookware .
keyword_replacements ['girl']
An Asian man sits on a stoop polishing cookware .
['an', 'asian', 'man', 'sits', 'on', 'a', 'st', '##oop', 'polish', '##ing', 'cook', '##ware', '.']
An Asian man sits on a stoop polishing cookware .
['an', 'asian', 'man', 'sits', 'on', 'a', 'st', '##oop', 'polish', '##ing', 'cook', '##ware', '.']
An Asian man sits on a stoop polishing cookware .
['an', 'asian',

predicted token  objects
ri -1 i 0 irresistibly printer.
methods -> objects
before:  Groups of young adults relax using various methods on a Greenspace .
after: Groups of young adults relax using various objects on a Greenspace .
keyword_replacements ['objects']
Groups of young adults relax using various methods on a Greenspace .
['groups', 'of', 'young', 'adults', 'relax', 'using', 'various', 'methods', 'on', 'a', 'greens', '##pace', '.']
A person wearing jeans and a t - shirt staining handrails on a sunny day .
['a', 'person', 'wearing', 'jeans', 'and', 'a', 't', '-', 'shirt', 'stain', '##ing', 'hand', '##rail', '##s', 'on', 'a', 'sunny', 'day', '.']
predicted token  someone
ri -1 i 0 irresistibly printer.
person -> someone
before:  A person wearing jeans and a t - shirt staining handrails on a sunny day .
after: A someone wearing jeans and a t - shirt staining handrails on a sunny day .
keyword_replacements ['someone']
A person wearing jeans and a t - shirt staining handrails on a s

In [22]:
list = []
for index, row in input_df.iterrows():
    if row['label']:
        list.append('yes')
    else: 
        list.append('no')
input_df.drop(columns=['label'])
input_df['label'] = list
print(input_df)

                                                context  \
0     A woman sells flowers and incense at the market .   
1      A woman sells flowers and fruits at the market .   
2     A woman sells flowers and incense at the market .   
3     well walking along a river near a beautiful ho...   
4     People walking along a river near a beautiful ...   
...                                                 ...   
4377  A someone is exiting a port - a - potty , reli...   
4378  A man is exiting a port - a - potty , relieved...   
4379  A someone is exiting a port - a - potty , reli...   
4380  A man is exiting a port - a - potty , relieved...   
4381  An asian man attempts to put out a wildfire wi...   

                                question               answer  start  end  \
0                   Who sells something?                 that      0    1   
1                What does someone sell?  flowers and incense      3    5   
2     Where does someone sell something?                 hom

In [23]:
copy = input_df.drop(columns=['start','end'])
print(copy)

                                                context  \
0     A woman sells flowers and incense at the market .   
1      A woman sells flowers and fruits at the market .   
2     A woman sells flowers and incense at the market .   
3     well walking along a river near a beautiful ho...   
4     People walking along a river near a beautiful ...   
...                                                 ...   
4377  A someone is exiting a port - a - potty , reli...   
4378  A man is exiting a port - a - potty , relieved...   
4379  A someone is exiting a port - a - potty , reli...   
4380  A man is exiting a port - a - potty , relieved...   
4381  An asian man attempts to put out a wildfire wi...   

                                question               answer label  
0                   Who sells something?                 that    no  
1                What does someone sell?  flowers and incense    no  
2     Where does someone sell something?                 home    no  
3          

In [24]:
copy.to_csv('data/QA_set_test2.csv', index=False)